## Image Loading

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import sklearn
from PIL import Image, ImageFilter, ImageEnhance

In [ ]:
!pip install pytesseract
!pip install tesseract
!sudo apt install tesseract-ocr
!which tesseract

In [ ]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd=r'/usr/bin/tesseract'

from PIL import Image
img=cv2.resize(cv2.imread("/content/img1.png"),(1200,500))
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
cv2.imwrite("temp.png",img)
plt.imshow(img)

## Node Text Recognition

In [ ]:
## extract all the text visible in the image
texts = pytesseract.image_to_string(img, config='-c tessedit_char_whitelist=ABCDEFGHIJKLM –psm 11', lang='eng')
print(texts)


In [ ]:
## draw boxes around character extracted to get their position
def draw_boxes_on_character(im,boxes):
    img_width = im.shape[1]
    img_height = im.shape[0]
    counter=0
    letters={}
    for box in boxes.splitlines():
        box = box.split(" ")
        if len(box):
          character = box[0]
          x = int(box[1])
          y = int(box[2])
          x2 = int(box[3])
          y2 = int(box[4])
          cv2.rectangle(im, (x, img_height - y), (x2, img_height - y2), (0, 255, 0), 1)
          counter+=1
          cv2.putText(im, str(counter)+" "+character, (x, img_height -y2), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 1)
          cv2.circle(im, ((x2+x)//2,img_height-(y2+y)//2), radius=5, color=(255, 100, 250), thickness=-1)
          letters[character]=[img_height-(y+y2)//2,(x+x2)//2]
    k=list(letters.keys())
    k.sort()
    letterS={i:letters[i] for i in k}
    return im,letterS
image= cv2.resize(cv2.cvtColor(cv2.imread("/content/temp.png"),cv2.COLOR_BGR2RGB),(1400,480))
im,letterS=draw_boxes_on_character(image,pytesseract.image_to_boxes(image, config='-c tessedit_char_whitelist=ABCDEFGHIJKLMN –psm 6', lang='eng'))
print(letterS)
plt.imshow(im)

## Vertices Recognition

In [ ]:
image= cv2.imread("/content/temp.png")
r,c,ch=image.shape
image=np.array(image).astype(np.uint8)

def vertex(img):
  v=np.zeros((r,c,ch)).astype(np.uint8)
  for i in range(r):
    for j in range(c):
      if img[i][j][2]==max(img[i][j][:]):
        v[i][j][0],v[i][j][1],v[i][j][2]=255,255,255
  return v
vertices=vertex(image)
vertices=cv2.cvtColor(vertices,cv2.COLOR_BGR2RGB)

## save the copy of image where vertices have been detected
cv2.imwrite("vertices.jpg",vertices)
plt.imshow(vertices)

In [ ]:
src=cv2.imread("vertices.jpg")
img = cv2.cvtColor(src, cv2.COLOR_RGB2GRAY)
img = cv2.blur(img, (3, 3))
cimg = src.copy() # numpy function

## through Hough Circle function in computer vision library, mark the vertices in circles
circles = cv2.HoughCircles(img, cv2.HOUGH_GRADIENT, 2, 10, np.array([]), 79, 23, 2, 26)
circles = np.uint16(np.around(circles))
if len(circles) != 0: # Check if circles have been found and only then iterate over these and add them to the image
  centers=[]
  counter=0
  _a,b,_c = circles.shape
  for i in range(b):
    cv2.circle(cimg, (circles[0][i][0], circles[0][i][1]), circles[0][i][2], (255, 0, 0), 3, cv2.LINE_AA)  # draw center of circle
    counter+=1
    cv2.putText(cimg, str(counter), (circles[0][i][0], circles[0][i][1]), cv2.FONT_HERSHEY_COMPLEX, 2, (0, 180, 250), 5)
    centers.append([circles[0][i][1],circles[0][i][0],circles[0][i][2]])
  print(centers)
plt.imshow(cimg)

In [ ]:
import math
def eucledian(x,y,x2,y2):
  return math.sqrt(((x2-x)**2)+((y2-y)**2))
Vcenters={}
for x,y,rad in centers:
  minDis=999999
  node=""
  for k,v in letterS.items():
    dis=eucledian(x,y,v[0],v[1])
    if minDis>dis:
      minDis=dis
      node=k
  Vcenters[node]=[x,y,rad]#[x,y],letterS[node],minDis]
k=list(Vcenters.keys())
k.sort()
VcenterS={i:Vcenters[i] for i in k}
VcenterS

## Edges Recognition

In [ ]:
def edge(img):
  e=np.zeros((r,c,ch)).astype(np.uint8)
  for i in range(r):
    for j in range(c):
      if img[i][j][0]==max(img[i][j][:]):
        e[i][j][0],e[i][j][1],e[i][j][2]=255,255,255
  return e
edges=edge(image)
edges=cv2.cvtColor(edges,cv2.COLOR_BGR2RGB)
cv2.imwrite('edges.jpg',edges)
plt.imshow(edges)

## Check Connectivity

In [ ]:
img1=vertices.copy()
img2=edges.copy()
def combineImg(img1,img2):
  rows,cols,channels = img2.shape
  roi = img1[0:rows, 0:cols ]
  gray = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
  ret, mask = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY_INV)
  mask_inv = cv2.bitwise_not(mask)
  graph = cv2.bitwise_and(roi,roi,mask = mask_inv)
  return graph
graph=combineImg(img1,img2)
plt.imshow(graph)

pixels, counts = np.unique(graph, return_counts=True)
pixCount=dict(zip(pixels, counts))
pixCount

## Adjacency Matrix

In [ ]:
import pandas as pd
n=len(VcenterS)
data=np.zeros((n,n)).astype(np.int8)
adjMatrix=pd.DataFrame(data,columns=VcenterS.keys(),index=VcenterS.keys())
edgeCount=0
edgeList={}
for node,cor in VcenterS.items():
  y,x,rad=cor[0],cor[1],cor[2]
  for k,v in VcenterS.items():
    if k==node or adjMatrix[node][k]>0:
      continue
    else:
      y2,x2,rad2=v[0],v[1],v[2]
      ewline=graph.copy()
      cv2.line( ewline, (x,y), (x2,y2), (0, 0, 0), thickness=1, lineType=8)
      newImg=combineImg(graph, ewline)
      p, c = np.unique(newImg, return_counts=True)
      pc=dict(zip(p, c))
      diff=abs(pixCount[0]-pc[0])

      if diff<100:
        adjMatrix[node][k]+=1
        adjMatrix[k][node]+=1
        edgeCount+=1
        edgeList[edgeCount]=[node,k]
adjMatrix # print adjacency matrix if needed

In [ ]:
print('Edges are :')
for k,v in edgeList.items():
  print(str(k)+" : "+v[0]+" to "+v[1])

## Incidence Matrix

In [ ]:
import pandas as pd
n=len(VcenterS)
m=edgeCount
data=np.zeros((n,m)).astype(np.int8)
incMatrix=pd.DataFrame(data,columns=[i for i in range(1,edgeCount+1)],index=VcenterS.keys())
for k,v in edgeList.items():
  for node in VcenterS.keys():
    if node in v:
      incMatrix[k][node]+=1
incMatrix # print incidence matrix if needed